In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/SSD/Code/TinyMPC/julia/bicycle_tvlqr/Project.toml`


In [2]:
using Symbolics
using LinearAlgebra
import ForwardDiff as FD

In [3]:
# ====================
# Explicit model params (working)
# State: x, y, theta
# Input: v, delta
# ====================
@variables x[1:3] u[1:2] 
x = collect(x); u = collect(u)
x0 = [19.4; 19.5; 0.676]; 
u0 = [-1.35; -0.0615]
# Dubins car
function dynamics(x, u)
    v = u[1]  # velocity
    δ = u[2]  # steering angle  

    θ = x[3]  # yaw

    c = cos(θ)
    s = sin(θ)

    ẋ = v*c
    ẏ = v*s
    θ̇ = v*tan(δ)
    return [ẋ, ẏ, θ̇]
end
function rk4(x, u)
    dt = 0.1
    k1 = dt*dynamics(x, u)
    k2 = dt*dynamics(x + k1/2, u)
    k3 = dt*dynamics(x + k2/2, u)
    k4 = dt*dynamics(x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end
function midpoint(x, u)
    dt = 0.1
    k1 = dt*dynamics(x, u)
    k2 = dt*dynamics(x + k1/2, u)
    x += k2
end

# sym_jac_A = Symbolics.jacobian(rk4(x, u), x)
# sym_jac_B = Symbolics.jacobian(rk4(x, u), u)
# Sub value to validate with ForwardDiff
# sub_dict = Dict()
# [sub_dict[x] = y for (x,y) in zip([x; u], [x0; u0])]
# val_jac_A = substitute.(sym_jac_A, (sub_dict,))
# val_jac_B = substitute.(sym_jac_B, (sub_dict,))
# fd_jac_A = FD.jacobian(_x->rk4(_x, u0), x0)
# fd_jac_B = FD.jacobian(_u->rk4(x0, _u), u0)
# display(norm(fd_jac_A - val_jac_A) + norm(fd_jac_B - val_jac_B))
# display(val_jac_A)
# display(val_jac_B)

midpoint (generic function with 1 method)

In [8]:
# ====================
# Generate C code for jacobians
# ====================
fA = build_function(sym_jac_A, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_Bicycle3dGetJacobianA_Raw, lhsname=:A, rhsnames=[Symbol("x"), Symbol("u")]);

open("functionA.txt","a") do io
        println(io, fA)
end

fB = build_function(sym_jac_B, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_Bicycle3dGetJacobianB_Raw, lhsname=:B, rhsnames=[Symbol("x"), Symbol("u")]);

open("functionB.txt","a") do io
        println(io, fB)
end

In [9]:
# ====================
# Generate C code for dynamics
# ====================
dynamics_rk4 = rk4(x ,u)
dynamics_rk4 = build_function(dynamics_rk4, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_Bicycle3dNonlinearDynamics_Raw, lhsname=:xn, rhsnames=[Symbol("x"), Symbol("u")])
open("dynamics_rk4.txt","a") do io
        println(io, dynamics_rk4)
end

In [ ]:
# ====================
# Symbolic model params
# not working 
# ====================
@variables g, m, ℓ, J
model = (g=g, m=m, ℓ=ℓ, J=J)
x = collect(x)
u = collect(u)
@variables x[1:6] u[1:2] 

# Planar Quadrotor Dynamics
function dynamics(model, x, u)
    θ = x[3]
    ẍ = (1/model.m)*(u[1] + u[2])*sin(θ)
    ÿ = (1/model.m)*(u[1] + u[2])*cos(θ) - model.g
    θ̈ = (1/model.J)*(model.ℓ/2)*(u[2] - u[1])
    
    return [x[4:6]; ẍ; ÿ; θ̈]
end
function rk4(model, x, u)
    dt = 0.1
    k1 = dt*dynamics(model, x, u)
    k2 = dt*dynamics(model, x + k1/2, u)
    k3 = dt*dynamics(model, x + k2/2, u)
    k4 = dt*dynamics(model, x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4) 
end
# sym_jac_A = Symbolics.jacobian(dynamics(model, x, u), x)
# sym_jac_B = Symbolics.jacobian(dynamics(model, x, u), u)
sym_jac_A = Symbolics.jacobian(rk4(model, x, u), u)

In [ ]:
function lqr_controller(t, x)
    #LQR Hover Controller
    A = FD.jacobian(_x->rk4(_x, u0), xref)
    B = FD.jacobian(_u->rk4(x0, _u), uref)
    K = dlqr(A,B,Q,R)
    return u_hover - K*(x-xref)
end

lqr_controller (generic function with 1 method)

In [ ]:
function closed_loop(x0,controller,N)
    xhist = zeros(length(x0),N)
    u0 = controller(1,x0)
    uhist = zeros(length(u0),N-1)
    uhist[:,1] .= u0
    xhist[:,1] .= x0
    for k = 1:(N-1)
        uk = controller(k,xhist[:,k], xref[k], uref[k])
        uhist[:,k] = max.(min.(umax, uk), umin) #enforce control limits
        xhist[:,k+1] .= rk4(xhist[:,k],uhist[:,k])
    end
    return xhist, uhist
end

closed_loop (generic function with 1 method)

In [4]:
nx = 3; nu = 2; N = 200;
uhist = [[1; 0.2] for i = 1:N-1]
xhist = [zeros(nx) for i = 1:N]
xhist[1] .= [0; 0; 0.0]
for k = 1:N-1
  xhist[k+1] = rk4(xhist[k], uhist[k])
end
display(xhist)
# export_vec_to_c("data", "sfloat", "U_ref_data", uhist)

200-element Vector{Vector{Float64}}:
 [0.0, 0.0, 0.0]
 [0.09999315158682012, 0.0010135154712834142, 0.02027100355086725]
 [0.19994521603621784, 0.00405364543211875, 0.0405420071017345]
 [0.2998151230934555, 0.00911914069458163, 0.06081301065260175]
 [0.39956183626222797, 0.01620791984912891, 0.081084014203469]
 [0.4991443696665392, 0.025317070119848864, 0.10135501775433625]
 [0.5985218048917784, 0.03644284856131937, 0.1216260213052035]
 [0.6976533077980773, 0.0495806835965824, 0.14189702485607075]
 [0.7964981452990376, 0.06472517689560273, 0.162168028406938]
 [0.8950157020989371, 0.08187010559343916, 0.18243903195780525]
 [0.9931654973815347, 0.10100842484721667, 0.2027100355086725]
 [1.090907201443618, 0.12213227073084898, 0.22298103905953975]
 [1.1882006522664592, 0.14523296346632192, 0.243252042610407]
 ⋮
 [-3.0609293051698185, 8.801840546505446, 3.810948667563052]
 [-3.1387172082770673, 8.739001866474332, 3.8312196711139195]
 [-3.215215414007845, 8.674599365763203, 3.85149067466478

In [29]:
x = [[1,2,3,4],[5,6,7,8]]

# Export a vector of vectors to C header and source 
function export_vec_to_c(filename, var_type, var_name, x)
    declare = var_type * " " * var_name * "[" * string(length(x)*length(x[1])) * "]"
    def = declare * " = {\n"
    for i=1:length(x)
        def = def * "  "
        for j=1:length(x[1])
            def = def * string(x[i][j])
            if j < length(x[1]) 
                def = def * ","
            end
        end
        if i < length(x)
            def = def * ",\n"
        end
    end
    def = def*"}"

    # open(filename*".h","a") do io
    #         println(io, declare * ";\n")
    # end
    open(filename*".h","a") do io
            println(io, def * ";\n")
    end
    return true
end
export_vec_to_c("data", "sfloat", "X_ref_data", xhist)

true

In [ ]:
function get_jacobians!(A, B, f, x0, u0)
    @assert size(A) == (Nx, Nx)
    @assert size(B) == (Nx, Nu)
    @assert size(f) == (Nx,)
    A .= FD.jacobian(_x->rk4(_x, u0), x0)
    B .= FD.jacobian(_u->rk4(x0, _u), u0)
    f .= rk4(x0, u0) - rk4(x0, u0)
end
function tiny_Riccati_LTV!(params, A::Matrix, B::Matrix, f::Vector, 
        get_jacobians::Function, X, U, K::Vector, d::Vector, P::Vector, p::Vector,μ,ρ)::Integer
    
    Q = params.Q; R = params.R; q = params.q; r = params.r; 
    xref = params.xref; uref = params.uref;
    Nx = params.Nx; Nu = params.Nu; N = params.N
    # Copy terminal cost-to-go
    k = N
    P[k] .= Q
    p[k] .= q

    # Sxx = zeros(n, n)
    # Sxu = zeros(n, m)
    # Sux = zeros(m, n)
    # Suu = zeros(m, m)
    # Sx = zeros(n, 1)
    # Su = zeros(m, 1)

    for k = N-1:-1:1 
        # get_jacobians!(A, B, f, X[k], U[k])

        Sx = q + A' * (P[k+1] * f + p[k+1])
        Su = r + B' * (P[k+1] * f + p[k+1])
        Sxx = Q + A'*P[k+1]*A
        Suu = R + B'*P[k+1]*B
        Sxu = P[k+1] * B                
        Suu = R + B'*P[k+1]*B                         
        Sux = B'*P[k+1]*A   
        
        # control constraints
        huv = ineq_con_u(U[k])  # calculate h(u) constraint
        mask = eval_mask(μ[k], huv)  # choose active
        ∇hu = ineq_con_u_jac(U[k])
        Su  += ∇hu'*(μ[k] + ρ*(mask * huv)) # add to cost
        Suu += ρ*∇hu'*mask*∇hu
        
        d[k] .= Suu\Su
        K[k] .= Suu\Sux

        P[k] .= Sxx + K[k]'*Suu*K[k] + K[k]'*Sux + Sux'*K[k]
        p[k] .= Sx + K[k]'*Suu*d[k] + K[k]'*Su + Sux'*d[k]
    end
    
    return 0;
end
function tiny_RiccatiForwardPass_LTV!(params, A, B, f, get_jacobians::Function,
                                    K::Vector, d::Vector, P::Vector, p::Vector, X::Vector, U::Vector) 
    N = params.N
    for k=1:N-1
        # get_jacobians!(A, B, f, X[k], U[k])
        U[k] .= -K[k]*X[k] - d[k]
        X[k+1] .= A*X[k] + B*U[k]
    end
  return 0
end
function tiny_AugmentedLagrange_LQR(params, A, B, f, get_jacobians, 
        K::Vector, d::Vector, P::Vector, p::Vector, X::Vector, U::Vector)
    # first check the sizes of everything
    @assert length(X) == params.N
    @assert length(U) == params.N-1
    @assert length(X[1]) == params.nx
    @assert length(U[1]) == params.nu
    @assert length(ineq_con_u(params,U[1])) == params.ncu  # no constraint control
    @assert length(ineq_con_x(params,X[1])) == params.ncx  # no constraint state    
function eval_mask(μv, huv)  
    # Extract active inequality constraints
    # active set mask
    mask = Diagonal(zeros(length(huv)))
    for i = 1:length(huv)
        mask[i,i] = (μv[i] > 0 || huv[i] > 0)
    end
    mask
end
function ineq_con_u(p,u)
    [u-p.u_max;-u + p.u_min]
end
function ineq_con_u_jac(params, u)
    FD.jacobian(_u -> ineq_con_u(params,_u), u)
end

eval_mask (generic function with 1 method)

In [ ]:
using DelimitedFiles
uref1 = readdlm("uref_data.txt")
uref1 = reshape(uref1, Nu, Nt-1)
xref1 = readdlm("xref_data.txt")
xref1 = reshape(xref1, Nx, Nt)
uref = [uref1[1:Nu, i] for i = 1:(Nt-1)]
xref = [xref1[1:Nx, i] for i = 1:Nt];
params = (
    Nx = Nx,
    Nu = Nu,
    Nt = Nt,
    N = Nh,
    xref = xref,
    uref = uref,
    Q = Q,
    R = R,
    q = q,
    r = r,
    u_min = umin,
    u_max = umax,
);

In [ ]:
dt = 0.1  
h = dt
Nx = 5     # number of state
Nu = 2     # number of controls
Tfinal = 10.0 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));
    
# Thrust limits
umax = [2; 0.5]
umin = -umax

# Cost weights
Q = Array(1.0*I(Nx)); 
q = zeros(Nx)
R = Array(.01*I(Nu)); 
r = zeros(Nu)
Qn = Array(1.0*I(Nx));

A = zeros(Nx, Nx)
B = zeros(Nx, Nu)
f = zeros(Nx)

x0 = [3; -1; 0; 1; 1]
xf = [20.0; 20.0; pi/4; 0; 0]


In [ ]:
xhist = [zeros(Nx) for i=1:Nt]
Phist = [zeros(Nx, Nx) for i=1:Nt]
phist = [zeros(Nx) for i=1:Nt]
uhist = [zeros(Nu) for i=1:Nt-1]
Khist = [zeros(Nu, Nx) for i=1:Nt-1]
dhist = [zeros(Nu) for i=1:Nt-1];
xhist = tiny_AL_LQR(params,X,U,P,p,K,d,Xn,Un;atol=1e-1,max_iters = 3000,verbose = true,ρ = 1e0, ϕ = 10.0);